In [7]:
OBJECT_NAMES  = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 
                 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 'bird', 
                 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 
                 'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 
                 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 
                 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 
                 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 
                 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cellone', 'microwave', 'oven', 
                 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 
                 'toothbrush']

In [12]:
# notebooks/infer_torch_frcnn.ipynb (dùng như script .py cũng được)
import torch, torchvision
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

# Load a font (replace with your font file path)
try:
    font = ImageFont.truetype("arial.ttf", 15)
except IOError:
    font = ImageFont.load_default()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="DEFAULT").to(device).eval()

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

img_path = "D:/Teach_n_Train/Advanced_Lessons_CV/LABS/r-cnn/faster-rcnn/traffic.jpg"
filename = os.path.basename(img_path)

img = Image.open(img_path).convert("RGB")
x = transform(img).to(device)
with torch.no_grad():
    out = model([x])[0]  # dict: boxes, labels, scores

boxes = out["boxes"].cpu().numpy()
labels = out["labels"].cpu().numpy()
scores = out["scores"].cpu().numpy()

draw = img.copy()
d = ImageDraw.Draw(draw)
for b,l,s in zip(boxes, labels, scores):
    if s < 0.5: continue
    x1,y1,x2,y2 = b
    d.rectangle([x1,y1,x2,y2], outline=(255,0,0), width=3)
    d.text((x1,y1), f"{OBJECT_NAMES[int(l)]}:{s:.2f}", font = font, fill=(255,255,0))

output_path = "D:/Teach_n_Train/Advanced_Lessons_CV/LABS/Object_Detection/Lab04/outputs"
os.makedirs(output_path, exist_ok=True)

draw.save(output_path + "/torch_infer_" + filename)
